In [76]:
import os
import requests,json
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from dotenv import load_dotenv
from openai import OpenAI
import ollama
import gradio as gr

In [13]:
load_dotenv(override=True)
api_key=os.getenv('OPENAI_API_KEY')

In [18]:
openai=OpenAI()

In [60]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
class Website:
    def __init__(self,url):
        self.url=url
        response=requests.get(url,headers=headers)
        soup=BeautifulSoup(response.content,'html.parser')
        self.title=soup.title
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text=soup.body.get_text(separator='\n',strip=True)
        else:
            self.text=""
        links=[link.get('href') for link in soup.find_all('a')]
        self.links=[link for link in links if link]
    def get_contents(self):
        return f"Webpage Title:\n{self.title} \n Webpage Content:\n{self.text}\n\n"
        

In [61]:
system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [62]:
def user_prompt(company_name,url):
    prompt =f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    return prompt

In [69]:
def boucher_gpt(company_name,url):
    response=openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
           { 'role':'system','content':system_message},
            {'role':'user','content':user_prompt(company_name,url)}
        ],
        stream = True,
    )
    result=""
    for chunk in response:
        result += chunk.choices[0].delta.content or ""
        yield result

In [70]:
boucher_gpt('Imaginist','https://www.imaginist.ai/')

<generator object boucher_gpt at 0x0000016087C55B40>

In [85]:
def broucher_ollama(company_name,url):
    response= ollama.chat(
        model='llama3.2',
        messages=[
           { 'role':'system','content':system_message},
            {'role':'user','content':user_prompt(company_name,url)}
        ],
        stream = True,

    )
    result = ""
    for chunk in response:
        result += chunk['message']['content'] or ""
        yield result




In [86]:
def stream_brouchure(company_name,url,model):
    if model == 'GPT':
        result=boucher_gpt(company_name,url)
    elif model == 'llama':
        result=broucher_ollama(company_name,url)
    else:
        raise ValueError("Unknown model")
    yield from result
        
    

In [87]:
view = gr.Interface(
    fn=stream_brouchure,
    inputs=[
        gr.Textbox(label="Company Name:", lines=6),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(['GPT','llama'],label="Choose model")
    ],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()


* Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.
